<a href="https://colab.research.google.com/github/60hr00t/draft1/blob/main/Vet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers datasets accelerate bitsandbytes sentencepiece fastapi uvicorn pyngrok nest_asyncio streamlit requests gTTS fuzzywuzzy python-Levenshtein streamlit-image-select streamlit-calendar pandas streamlit-mic-recorder openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 15.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.7/706.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 87.5 MB/s eta 0:00:0

In [3]:
# Detecting CUDA GPU with PyTorch
import torch, platform

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

In [4]:
# ==========================================================
# Upload a CSV with “prompt” and “response” columns in Colab
# and set `CSV_PATH` to the selected file.
# ==========================================================

import io, pandas as pd
from google.colab import files

print("Upload a CSV with columns: prompt,response")
uploaded = files.upload()
if uploaded:
    fname = list(uploaded.keys())[0]
    CSV_PATH = f"/content/{fname}"
    print("Using uploaded file:", CSV_PATH)


Upload a CSV with columns: prompt,response


Saving Vet.csv - Vet.csv to Vet.csv - Vet.csv
Using uploaded file: /content/Vet.csv - Vet.csv


In [5]:
# ==================================================
# Fuzzy match answers from CSV and render as plain
# text or option buttons
# ==================================================
from fuzzywuzzy import fuzz
df = pd.read_csv(CSV_PATH)
required = {"prompt", "response"}
assert required.issubset(df.columns), f"CSV must include: {required}. Found: {df.columns.tolist()}"
print(f"Loaded {len(df)} rows.")

# This function returns the best-matching response string for user_msg
# or None if the best score is below threshold.
def answer_from_csv(user_msg, df, threshold=90):
    best_score = 0
    best_response = None
    for _, row in df.iterrows():
        score = fuzz.partial_ratio(user_msg.lower(), row["prompt"].lower())
        if score > best_score:
            best_score = score
            best_response = row["response"]
    return best_response if best_score >= threshold else None
     # Check if response contains semicolons
    if ";" in response:
        options = [opt.strip() for opt in response.split(";")]
        st.markdown(f"**{prompt.capitalize()} Options:**")
    for i, option in enumerate(options):
            if st.button(option, key=f"{prompt}_{i}"):
                st.chat_message("user").markdown(option)
                reply = get_bot_response(option)  # Send to your backend
                st.chat_message("assistant").markdown(reply)
    else:
# If no semicolon, show as regular response
        st.markdown(f"**{prompt.capitalize()}:** {response}")




Loaded 2 rows.


In [6]:
# =====================================================================
# This snippet downloads and initializes Google’s FLAN-T5 (small) model
# and its tokenizer using the  Transformers library, then puts the model
# in evaluation mode for inference.
# =====================================================================
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [7]:
# ==========================================================================
# This section allows the FastAPI app exposes a chat endpoint that first looks
# up a reply in a CSV via answer_from_csv, falls back to a generative model
# if no strong match exists, synthesizes the reply to MP3 with gTTS, and serves
# the audio from a static /audio path.
# ==========================================================================
from fastapi import FastAPI
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
import uvicorn, nest_asyncio, threading, time, uuid, os
from pyngrok import conf, ngrok
from gtts import gTTS

app = FastAPI()
app.mount("/audio", StaticFiles(directory="/content", html=True), name="audio")

class Prompt(BaseModel):
    prompt: str

@app.post("/respond")
async def respond(request: Prompt):
    user_msg = request.prompt.strip()
    csv_answer = answer_from_csv(user_msg, df, threshold=90)
    filename = f"audio_{uuid.uuid4().hex[:8]}.mp3"
    filepath = f"/content/{filename}"

    if csv_answer:
        gTTS(text=csv_answer, lang='en').save(filepath)
        return {"response": csv_answer, "audio_path": f"/audio/{filename}"}

    prompt_text = f"You are a helpful cultural assistant. Answer clearly.\nUser: {user_msg}\nAssistant:"
    input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids
    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=120)
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    gTTS(text=reply, lang='en').save(filepath)
    return {"response": reply, "audio_path": f"/audio/{filename}"}

@app.get("/audio/{filename}")
async def get_audio(filename: str):
    return FileResponse(path=f"/content/{filename}", media_type="audio/mpeg")


In [10]:
# =====================================================================
# This snippet runs the FastAPI app with Uvicorn in a background thread
# (using nest_asyncio), opens an ngrok tunnel with the auth token, then
# builds and prints the public /respond endpoint (FASTAPI_URL) and the
# base URL for serving audio (AUDIO_BASE_URL).
# ======================================================================
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

nest_asyncio.apply()
threading.Thread(target=run_api, daemon=True).start()
time.sleep(5)

conf.get_default().auth_token = "36Jyi3nI1rdAxQw3UOMT4SrCerI_3DHRyYHWaKb9PsJorLM3n"
public_url = ngrok.connect(8000)
FASTAPI_URL = f"{public_url.public_url}/respond"
AUDIO_BASE_URL = public_url.public_url
print("FASTAPI URL:", FASTAPI_URL)
print("AUDIO BASE:", AUDIO_BASE_URL)

INFO:     Started server process [148]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/thr

FASTAPI URL: https://unvibrated-tubercularly-darwin.ngrok-free.dev/respond
AUDIO BASE: https://unvibrated-tubercularly-darwin.ngrok-free.dev


In [11]:
import os
os.environ["SMTP_HOST"]="smtp.gmail.com"
os.environ["SMTP_PORT"]="587"
os.environ["SMTP_USER"]=""
os.environ["SMTP_PASS"]="your_app_password"
os.environ["FROM_EMAIL"]=""

Overwriting vet_finder_app.py


In [51]:
# Kill any existing Streamlit process safely
!fuser -k 8501/tcp 2>/dev/null || true

import time

# Give the port time to clear
time.sleep(3)

# --- Run Streamlit App ---
print("🚀 Starting Streamlit...")
!streamlit run vet_finder_app.py --server.headless true --server.port 8501 > /content/streamlit_log.txt 2>&1 &

# Wait for Streamlit to start
time.sleep(8)

# --- Connect ngrok ---
from pyngrok import ngrok

# Kill previous tunnels
ngrok.kill()

# Start new tunnel
public_url = ngrok.connect(8501).public_url

print("✅ Streamlit available at:", public_url)


 16930

🚀 Starting Streamlit...


✅ Streamlit available at: https://unvibrated-tubercularly-darwin.ngrok-free.dev
